---

# Data Mining:<br>Statistical Modeling and Learning from Data

## Dr. Ciro Cattuto<br>Dr. Laetitia Gauvin<br>Dr. André Panisson

### Exercises - Text Message Spam Filter

---

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Building a Text Message Spam Filter with a Naive Bayes Classifier

This tutorial explains how to classify text messages as spam / not spam using scikit-learn.

# Download Dataset
The SMS Spam Collection is open source and available at the UCI Machine Learning Repository.
The data files and documentation can be found here: http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

## Reading the Dataset into Memory

In [2]:
messages = []
categories = []
for line in open("data/smsdata.txt"):
    category, message = line.split('\t')
    messages.append(message)
    categories.append(category)

y = array([0 if item=="ham" else 1 for item in categories])
 
print()
print (" %d Not Spam" % (y==0).sum())
print ("+ %d Spam" % (y==1).sum())
print (" ---------")
print (" %d Total" % len(y))
print() 
print ("Proportion spam: %.2f/100" % (100.*(y==1).sum() / float(len(y))))

FileNotFoundError: ignored

# From Text Messages to Feature Vectors
We need to transform our text data into feature vectors, numerical representations which are suitable for performing statistical analysis. The most common way to do this is to apply a bag-of-words approach where the frequency of an occurrence of a word becomes a feature for our classifier.


## Term Frequency-Inverse Document Frequency

We want to consider the relative importance of particular words, so we'll use term frequency–inverse document frequency as a weighting factor. This will control for the fact that some words are more "spamy" than others.

## Mathematical details

tf–idf is the product of two statistics, term frequency and inverse document
frequency. Various ways for determining the exact values of both statistics
exist. In the case of the '''term frequency''' tf(''t'',''d''), the simplest
choice is to use the ''raw frequency'' of a term in a document, i.e. the
number of times that term ''t'' occurs in document ''d''. If we denote the raw
frequency of ''t'' by f(''t'',''d''), then the simple tf scheme is
tf(''t'',''d'') = f(''t'',''d''). Other possibilities
include:

  * boolean_data_type "frequencies": tf(''t'',''d'') = 1 if ''t'' occurs in ''d'' and 0 otherwise; 
  * logarithmically scaled frequency: tf(''t'',''d'') = log (f(''t'',''d'') + 1); 
  * augmented frequency, to prevent a bias towards longer documents, e.g. raw frequency divided by the maximum raw frequency of any term in the document: :$\mathrm{tf}(t,d) = 0.5 + \frac{0.5 \times \mathrm{f}(t, d)}{\max\{\mathrm{f}(w, d):w \in d\}}$

The '''inverse document frequency''' is a measure of whether the term is
common or rare across all documents. It is obtained by dividing the total
number of documents by the number of documents containing the
term, and then taking the logarithm of that quotient.

:$\mathrm{idf}(t, D) = \log \frac{|D|}{|\{d \in D: t \in d\}|}$

with

  * $|D|$: cardinality of D, or the total number of documents in the corpus 
  * $|\{d \in D: t \in d\}|$ : number of documents where the term $ t $ appears (i.e., $\mathrm{tf}(t,d) eq 0$). If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the formula to $1 + |\{d \in D: t \in d\}|$. 

Mathematically the base of the log function does not matter and constitutes a
constant multiplicative factor towards the overall result.

Then tf–idf is calculated as

$$\mathrm{tfidf}(t,d,D) = \mathrm{tf}(t,d) \times \mathrm{idf}(t, D)$$

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

pattern ='(?u)\\b[A-Za-z]{3,}'

cv = CountVectorizer(stop_words=None, token_pattern=pattern,
                     ngram_range=(1, 3))
C = cv.fit_transform(messages)

tfidf = TfidfTransformer(sublinear_tf=True)
#tfidf = TfidfVectorizer(sublinear_tf=True, max_df=0.5)
                        
#calculate features using tf-idf and create a training set 
X_train = tfidf.fit_transform(C)
print ()
print ("X_train is a sparse matrix with shape: %s" % str(X_train.shape))
print()

Here, we create a variable, tfidf, which is a vectorizer responsible for performing three important steps:

- First, it will build a dictionary of features where keys are terms and values are indices of the term in the feature matrix (that's the fit part in fit_transform)
- Second, it will transform our documents into numerical feature vectors according to the frequency of words appearing in each text message. Since any one text message is short, each feature vector will be made up of mostly zeros, each of which indicates that a given word appeared zero times in that message.
- Lastly, it will compute the tf-idf weights for our term frequency matrix.

# Naive Bayes

Using Bayes' theorem, the conditional probability of observing a class $C_k$ given that we observed a set of features $\mathbf{x}$ can be decomposed as

$$p(C_k \mid \mathbf{x}) = \frac{p(C_k) \ p(\mathbf{x} \mid C_k)}{p(\mathbf{x})}$$.

### Bank example

PC(Previous Client), CR(Criminal Record), Age, MP(Missed Payments), Res(Result of classification)

<table border="0" cellpadding="0" cellspacing="0">
    <colgroup>
        <col width="103">
        <col width="103">
        <col width="103">
        <col width="103">
        <col width="103">
        <col width="103">
    </colgroup>

    <tbody>
        <tr>
            <td>
                <p>Id</p>
            </td>

            <td>
                <p>PC</p>
            </td>

            <td>
                <p>CR</p>
            </td>

            <td>
                <p>Age</p>
            </td>

            <td>
                <p>MP</p>
            </td>

            <td>
                <p>Res</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>1</p>
            </td>

            <td>
                <p>1</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>50</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>E</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>2</p>
            </td>

            <td>
                <p>1</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>28</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>E</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>3</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>35</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>E</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>4</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>55</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>E</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>5</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>49</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>E</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>6</p>
            </td>

            <td>
                <p>1</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>75</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>NE</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>7</p>
            </td>

            <td>
                <p>1</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>38</p>
            </td>

            <td>
                <p>10</p>
            </td>

            <td>
                <p>NE</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>8</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>83</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>NE</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>9</p>
            </td>

            <td>
                <p>1</p>
            </td>

            <td>
                <p>1</p>
            </td>

            <td>
                <p>44</p>
            </td>

            <td>
                <p>5</p>
            </td>

            <td>
                <p>NE</p>
            </td>
        </tr>

        <tr>
            <td>
                <p>10</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>1</p>
            </td>

            <td>
                <p>28</p>
            </td>

            <td>
                <p>0</p>
            </td>

            <td>
                <p>NE</p>
            </td>
        </tr>
    </tbody>
</table>

Given that 

$$p(\mathbf{x} \mid C_k)  =  \prod_{i=1}^{D} p(x_i \mid C_k) $$

then we have that

$$p(C_k \mid \mathbf{x}) \propto p(C_k) \ \prod_{i=1}^{D} p(x_i \mid C_k) $$

and then we choose the predicted class by:

$$\hat{y} = \arg\max_{C_k} p({C_k}) \prod_{i=1}^{D} p(x_i \mid {C_k})$$

For example, to predict the class for the following cases:

<table border="0" cellpadding="0" cellspacing="0" class="Table2">
    <colgroup>
        <col width="103">
        <col width="103">
        <col width="103">
        <col width="103">
        <col width="103">
        <col width="103">
    </colgroup>

    <tr class="Table21">
        <td>
            <p class="P4">Id</p>
        </td>

        <td>
            <p class="P4">PC</p>
        </td>

        <td>
            <p class="P4">CR</p>
        </td>

        <td>
            <p class="P4">Age</p>
        </td>

        <td>
            <p class="P4">MP</p>
        </td>

        <td>
            <p class="P4">Res</p>
        </td>
    </tr>

    <tr class="Table21">
        <td>
            <p class="P3">11</p>
        </td>

        <td>
            <p class="P3">0</p>
        </td>

        <td>
            <p class="P3">0</p>
        </td>

        <td>
            <p class="P3">50</p>
        </td>

        <td>
            <p class="P3">0</p>
        </td>

        <td>
            <p class="P4">?</p>
        </td>
    </tr>

    <tr class="Table21">
        <td>
            <p class="P3">12</p>
        </td>

        <td>
            <p class="P3">1</p>
        </td>

        <td>
            <p class="P3">0</p>
        </td>

        <td>
            <p class="P3">80</p>
        </td>

        <td>
            <p class="P3">0</p>
        </td>

        <td>
            <p class="P4">?</p>
        </td>
    </tr>
</table>

From:

- $\mathbf{C} \in \{\mathbf{E}, \mathbf{NE}\}$, $p(\mathbf{E}) = 1/2$, $p(\mathbf{NE}) = 1/2$


- $\mathbf{PC} \in \{0, 1\}$, $p(0 \mid \mathbf{E}) = 3/5$, $p(1 \mid \mathbf{E}) = 2/5$, $p(0 \mid \mathbf{NE}) = 2/5$, $p(1 \mid \mathbf{NE}) = 3/5$


- $\mathbf{CR} \in \{0, 1\}$, $p(0 \mid \mathbf{E}) = 1$, $p(1 \mid \mathbf{E}) = 0$, $p(0 \mid \mathbf{NE}) = 3/5$, $p(1 \mid \mathbf{NE}) = 2/5$


- $\mathbf{Age} \in \{\lt 65, \gt 65 \}$, $p(\lt 65 \mid \mathbf{E}) = 1$, $p(\gt 65 \mid \mathbf{E}) = 0$, $p(\lt 65 \mid \mathbf{NE}) = 3/5$, $p(\gt 65 \mid \mathbf{NE}) = 2/5$


- $\mathbf{MP} \in \{0, \gt 0 \}$, $p(0 \mid \mathbf{E}) = 1$, $p(\gt 0 \mid \mathbf{E}) = 0$, $p(0 \mid \mathbf{NE}) = 3/5$, $p(\gt 0 \mid \mathbf{NE}) = 2/5$

For object 11, we have that:

$p(\mathbf{E} \mid [ 0, 0, \lt 65, 0] =  1/2 \times 3/5 \times 1 \times 1 \times 1 = 0.3$

$p(\mathbf{NE} \mid [ 0, 0, \lt 65, 0] = 1/2 \times 2/5 \times 3/5 \times 3/5 \times 3/5 = 0.04$

Naive Bayes can be modeled in several different ways. For example, to model a feature using a **normal** density function, we have that:

$$p(x_i \mid C_k)  =  \frac{1}{{\sigma_{ik} \sqrt {2\pi } }} e^{{ - \left( {x - \mu_{ik} } \right)^2 } / {2\sigma_{ik} ^2 }} $$

and to model a feature using a **multinomial** density function, we have that the multinomial naive Bayes classifier becomes a linear classifier when expressed in log-space:

\begin{align}
\log p(C_k|\mathbf{x}) & \varpropto \log \left( p(C_k) \prod_{i=1}^n {p_{ki}}^{x_i} \right) \\
                       & = \log p(C_k) + \sum_{i=1}^n x_i \cdot \log p_{ki}                 \\
                       & = b + \mathbf{w}_k^\top \mathbf{x}
\end{align}

where $b = \log p(C_k)$ and $w_{ki} = \log p_{ki}$.


---

We'll be using SciKits' MultinomialNB, a Naive Bayes classifier effective for catching spam with the added benefits of scalability and low training time.

MultinomialNB implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification (where the data are typically represented as word vector counts, although tf-idf vectors are also known to work well in practice). The distribution is parametrized by vectors $\mathbf{w}_y = ( w_{y1},\ldots, w_{yD})$ for each class $y$, where $D$ is the number of features (in text classification, the size of the vocabulary) and $w_{yi}$ is the probability $P(x_i \mid y)$ of feature $i$ appearing in a sample belonging to class $y$.
The parameters $\mathbf{w}_y$ are estimated by a smoothed version of maximum likelihood, i.e. relative frequency counting:
$w_{yi} = \frac{ N_{yi} + \alpha}{N_y + \alpha n}$
where $N_{yi} = \sum_{x \in T} x_i$ is the number of times feature $i$ appears in a sample of class $y$ in the training set $T$, and $N_{y} = \sum_{i=1}^{|T|} N_{yi}$ is the total count of all features for class $y$.
The smoothing priors $\alpha \ge 0$ accounts for features not present in the learning samples and prevents zero probabilities in further computations. Setting $\alpha = 1$ is called *Laplace smoothing*, while $\alpha < 1$ is called *Lidstone smoothing*.

**Why Multinomial Naive Bayes**?   
Each row of the training set represents a document. A document is a list of $n$ words. If we consider that each word $w_i$ of the vocabulary appears in the vocabulary with probability $p_i$, then each document can be represented as a **multinomial distribution** with $n$ trials. The number of possible outcomes is the number of words in the vocabulary, and in each trial we choose a word from the vocabulary following the event probabilities $p_i$.

In [0]:
#create a list of training labels. 1 is spam, 0 if ham
y_train = y
 
print ("y_train is a list of categories: %s ..." % str(y_train)[:70])
print ("X_train has %d feature vectors" % (X_train.shape[0]))
print ("y_train has %d target classes" %(len(y_train)))
print ("dataset has %d rows" %(len(messages)))
print
 
# create a Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
 
clf.fit(X_train, y_train)
print ("Trained MultinomialNB Classifier")
print ("Coefficients: %s ..." % (str(clf.coef_)[:70]))
print ("   Intercept: %s" %(str(clf.intercept_)))

We create a variable, y_train, which is simply a list of target classes which our classifier will be trained to identify. 1 indicates spam while 0 indicates ham, or non-spam.

Then we fit the model by passing the X_train sparse matrix and y_train to our MultinomialNB classifier's fit function.

#Classifying New Observations

Now let's classify the test documents as spam or not spam and see how we did.

In [0]:
test_messages = ["Call MobilesDirect free on 08000938767 to update now! or2stoptxt",
                 "Call now for a free trial offer!",
                 "Hey Sam, want to get some pizza later?",
                 "idk my bff jill?",
                 "Free later for a beer? Call me now!"]
                 
# extract features from raw text documents
C_test = cv.transform(test_messages)
X_test = tfidf.transform(C_test)
 
# MultinomialNB's predict classes directly
print ("Classified: %s" % clf.predict(X_test))

The predict function yields an array of True / False values (True for spam, False for not spam).

## Exercise:

The classification results for the test documents are not very encouraging.   
Find the best parameter for the MultinomialNB model, and check the classification results for the test documents.